# Equality of Odds
Equalized odds postprocessing is a post-processing technique that solves a linear program to find probabilities with which to change output labels to optimize equalized odds.

**References**
* M. Hardt, E. Price, and N. Srebro, “Equality of Opportunity in Supervised Learning,” Conference on Neural Information Processing Systems, 2016.
* G. Pleiss, M. Raghavan, F. Wu, J. Kleinberg, and K. Q. Weinberger, “On Fairness and Calibration,” Conference on Neural Information Processing Systems, 2017.

In [1]:
# import relevant dependencies
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from aif360.datasets import StandardDataset
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.metrics import ClassificationMetric

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

In [4]:
def create_dataset(
    X: pd.DataFrame,
    y,
    protected_attribute_name: str
) -> StandardDataset:
    if isinstance(y, np.ndarray):
        y = pd.Series(y.flatten(), index=X.index, name='class')
    return StandardDataset(
        df=pd.concat([X, y], axis=1),
        label_name="class",
        favorable_classes=[1],
        protected_attribute_names=[protected_attribute_name],
        privileged_classes=[[1]],
    )


In [5]:
# fetch raw-data from sklearn.datasets
raw_data = fetch_openml(data_id=1590, as_frame=True)

c:\Users\I2044\Anaconda3\envs\python38\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [6]:
from sklearn.preprocessing import MinMaxScaler

X_raw = pd.get_dummies(raw_data.data)
X_raw = pd.DataFrame(MinMaxScaler().fit_transform(X_raw), columns=X_raw.columns)
y = 1 * (raw_data.target == ">50K")

X_train, X_test, y_train, y_test = train_test_split(X_raw, y, test_size=0.5, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [7]:
LR = LogisticRegression(solver="liblinear", random_state=42)
LR.fit(X_train, y_train)

y_val_pred = LR.predict(X_val).reshape(-1,1)
y_val_pred_proba = LR.predict_proba(X_val)[:,1].reshape(-1,1)
y_test_pred = LR.predict(X_test).reshape(-1,1)
y_test_pred_proba = LR.predict_proba(X_test)[:,1].reshape(-1,1)

In [8]:
protected_attribute_name = "sex_Male"

privileged_groups = [{protected_attribute_name: 1.0}]
unprivileged_groups = [{protected_attribute_name: 0.0}]

In [9]:
dataset_val = create_dataset(X_val, y_val, protected_attribute_name)
dataset_val_pred = create_dataset(X_val, y_val_pred, protected_attribute_name)
dataset_val_pred.scores = y_val_pred_proba

EOP = EqOddsPostprocessing(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)
EOP.fit(dataset_val, dataset_val_pred)

In [10]:
dataset_test = create_dataset(X_test, y_test, protected_attribute_name)
dataset_test_pred = create_dataset(X_test, y_test_pred, protected_attribute_name)

dataset_test_pred_transf = EOP.predict(dataset_test_pred)

In [11]:
metric_unmit = ClassificationMetric(
    dataset_test,
    dataset_test_pred,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

metric_mit = ClassificationMetric(
    dataset_test,
    dataset_test_pred_transf,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [12]:
metric_unmit.disparate_impact()

0.28442441174103594

In [13]:
metric_mit.disparate_impact()

0.6382341345377526

In [14]:
metric_unmit.accuracy()

0.8517608517608518

In [15]:
metric_mit.accuracy()

0.8177723177723177